## FRAUD DETECTION
#### THIS WORK IS DONE BY PRATYUSH SETHI

In [1]:
import numpy as np
import tensorflow as tf
import json
import pandas as pd
import math
import os
import time
import spacy
import random
from DATASETS import ques_triples
import csv

# importing torch libraries
import torch
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch.nn as nn
import torch.optim as optim
import torchtext as tt
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer
from collections import Counter
import io
from torchtext.vocab import Vocab

#Importing machine learning libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.decomposition import FastICA
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import precision_recall_fscore_support

In C:\Users\Praty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Praty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Praty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Praty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Praty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
# READING FILE transactions renamed as DATA_TRANS
# change the file path
dim = []
def parse(d):
    dictionary = dict()
    # Removes curly braces and splits the pairs into a list
    pairs = d.strip('{}').split(', ')
    for i in pairs:
        pair = i.split(': ')
        # Other symbols from the key-value pair should be stripped.
        dictionary[pair[0].strip('\'\'\"\"')] = pair[1].strip('\'\'\"\"')
    return dictionary
try:
    geeky_file = open('C:/Users/Praty/OneDrive/Desktop/DATA_TRANS.txt', 'rt')
    lines = geeky_file.read().split('\n')
    for l in lines:
        if l != '':
            dim.append(parse(l))
            #print(dictionary)
    geeky_file.close()
except:
    print("Something unexpected occurred!")

In [3]:
dfItem = pd.DataFrame.from_records(dim)

In [4]:
dfItem.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000.0,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,,0.0,,,,false,,,false,false
1,737265056,737265056,5000.0,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,,0.0,,,,true,,,false,false
2,737265056,737265056,5000.0,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,,0.0,,,,false,,,false,false
3,737265056,737265056,5000.0,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,,0.0,,,,false,,,false,false
4,830329091,830329091,5000.0,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,,0.0,,,,true,,,false,false


#### Preprocessing step
#### 1. checking for null values
#### 2. checking for data types
#### 3. checking count

In [6]:
print(dfItem.dtypes)
print(dfItem.count())
print(dfItem.isnull().sum())

accountNumber               object
customerId                  object
creditLimit                 object
availableMoney              object
transactionDateTime         object
transactionAmount           object
merchantName                object
acqCountry                  object
merchantCountryCode         object
posEntryMode                object
posConditionCode            object
merchantCategoryCode        object
currentExpDate              object
accountOpenDate             object
dateOfLastAddressChange     object
cardCVV                     object
enteredCVV                  object
cardLast4Digits             object
transactionType             object
echoBuffer                  object
currentBalance              object
merchantCity                object
merchantState               object
merchantZip                 object
cardPresent                 object
posOnPremises               object
recurringAuthInd            object
expirationDateKeyInMatch    object
isFraud             

In [7]:
# Fetching only those which are not empty coulmns and only those which are required
df = dfItem[['creditLimit','availableMoney','transactionAmount',
             'posEntryMode','posConditionCode',
             'cardCVV','enteredCVV','cardLast4Digits',
             'currentBalance','cardPresent',
             'expirationDateKeyInMatch','isFraud']]

In [8]:
df

,creditLimit,availableMoney,transactionAmount,posEntryMode,posConditionCode,cardCVV,enteredCVV,cardLast4Digits,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,5000.0,5000.0,98.55,02,01,414,414,1803,0.0,false,false,false
1,5000.0,5000.0,74.51,09,01,486,486,767,0.0,true,false,false
2,5000.0,5000.0,7.47,09,01,486,486,767,0.0,false,false,false
3,5000.0,5000.0,7.47,09,01,486,486,767,0.0,false,false,false
4,5000.0,5000.0,71.18,02,01,885,885,3143,0.0,true,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...
786358,50000.0,48904.96,119.92,90,01,936,936,3783,1095.04,false,false,false
786359,50000.0,48785.04,18.89,09,01,939,939,3388,1214.96,false,false,false
786360,50000.0,48766.15,49.43,02,01,936,936,3783,1233.85,false,false,false
786361,50000.0,48716.72,49.89,09,99,939,939,3388,1283.28,false,false,false


In [9]:
print(np.isnan(df.any())) #and gets False
print(np.isfinite(df.all())) #and gets True

creditLimit                 False
availableMoney              False
transactionAmount           False
posEntryMode                False
posConditionCode            False
cardCVV                     False
enteredCVV                  False
cardLast4Digits             False
currentBalance              False
cardPresent                 False
expirationDateKeyInMatch    False
isFraud                     False
dtype: bool
creditLimit                 True
availableMoney              True
transactionAmount           True
posEntryMode                True
posConditionCode            True
cardCVV                     True
enteredCVV                  True
cardLast4Digits             True
currentBalance              True
cardPresent                 True
expirationDateKeyInMatch    True
isFraud                     True
dtype: bool


In [10]:
# replacing all false to 0 and true to 1 and blanks to 0.
df.replace("false", 0,inplace = True) 
df.replace("true",1,inplace = True)
df.replace(" ","0",inplace = True)

C:\Users\Praty\Anaconda3\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [15]:
# stripping the 0's in column posEntryMode and posConditionCode

df['posEntryMode'] = df['posEntryMode'].str.lstrip('0')
df['posConditionCode'] = df['posConditionCode'].str.lstrip('0')

# changing all the datatypes from object to float

df["creditLimit"] = df["creditLimit"].astype(str).astype(float)
df["availableMoney"] = df["availableMoney"].astype(str).astype(float)
df["transactionAmount"] = df["transactionAmount"].astype(str).astype(float)
df["cardCVV"] = df["cardCVV"].astype(str).astype(float)
df["enteredCVV"] = df["enteredCVV"].astype(str).astype(float)
df["cardLast4Digits"] = df["creditLimit"].astype(str).astype(float)
df["currentBalance"] = df["currentBalance"].astype(str).astype(float)
df['cardPresent'] = df['cardPresent'].astype(str).astype(float)
df['expirationDateKeyInMatch'] = df['expirationDateKeyInMatch'].astype(str).astype(float)
df['isFraud'] = df['isFraud'].astype(str).astype(float)
df['posEntryMode'] = pd.to_numeric(df['posEntryMode'])
df['posConditionCode'] = pd.to_numeric(df['posConditionCode'])

C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praty\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [16]:
df.dtypes

creditLimit                 float64
availableMoney              float64
transactionAmount           float64
posEntryMode                float64
posConditionCode            float64
cardCVV                     float64
enteredCVV                  float64
cardLast4Digits             float64
currentBalance              float64
cardPresent                 float64
expirationDateKeyInMatch    float64
isFraud                     float64
dtype: object

In [17]:
df.head()

,creditLimit,availableMoney,transactionAmount,posEntryMode,posConditionCode,cardCVV,enteredCVV,cardLast4Digits,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,5000.0,5000.0,98.55,2.0,1.0,414.0,414.0,5000.0,0.0,0.0,0.0,0.0
1,5000.0,5000.0,74.51,9.0,1.0,486.0,486.0,5000.0,0.0,1.0,0.0,0.0
2,5000.0,5000.0,7.47,9.0,1.0,486.0,486.0,5000.0,0.0,0.0,0.0,0.0
3,5000.0,5000.0,7.47,9.0,1.0,486.0,486.0,5000.0,0.0,0.0,0.0,0.0
4,5000.0,5000.0,71.18,2.0,1.0,885.0,885.0,5000.0,0.0,1.0,0.0,0.0


In [18]:
# MACHINE LEARNING MODEL AS ITS A BINARY CLASSIFICATION SO USING LOGISTIC REGRESSION
X_FEATURES = ['creditLimit','availableMoney','transactionAmount','posEntryMode',
           'posConditionCode','cardCVV','enteredCVV',
           'cardLast4Digits','currentBalance','cardPresent','expirationDateKeyInMatch']
x_features = df.loc[:,X_FEATURES].values.astype(int)
y_features = df.loc[:,['isFraud']].values.astype(int)

In [19]:
X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(x_features, y_features, test_size=0.20, random_state = 101)

In [21]:
#LOGISTIC REGRESSION IMPLEMENTATION
from sklearn.linear_model import LogisticRegression

Logistic_Reg = LogisticRegression(penalty = 'l2', tol = 0.0001, C = 1.0)
Logistic_Reg.fit(X_Train, Y_Train)
print("LOGISTIC REGRESSION :", Logistic_Reg.score(X_Test, Y_Test))

Y_PREDICTION = Logistic_Reg.predict(X_Test)
print("ACCURACY OF MODEL :", metrics.accuracy_score(Y_Test, Y_PREDICTION))

#USING CROSS VALIDATION

scores = cross_val_score(Logistic_Reg, x_features, y_features, cv = 5)
print("CROSS-V:", scores.mean())

C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for ex

LOGISTIC REGRESSION : 0.9842503163289312
ACCURACY OF MODEL : 0.9842503163289312


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max

CROSS-V: 0.9842095826036654


In [31]:
print("recall precision f1-score")
print(precision_recall_fscore_support(Y_Test, Y_PREDICTION, average='weighted'))

recall precision f1-score
(0.9687486851936011, 0.9842503163289312, 0.9764379798471058, None)


C:\Users\Praty\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
# GETTING HIGH PRECISION AND RECALL INDICATE THAT FALSE NEGATIVE
# AND FALSE POSITIVE ARE LESS IN THE DATA.